In [4]:
# Tangent Kernel Feature Debugging Notebook

import numpy as np
import torch
import os
import glob

# ---- Utility Functions ----
def summarize_numpy_stats(name, array):
    print(f"[NumPy] {name} -> min: {np.min(array):.4f}, max: {np.max(array):.4f}, mean: {np.mean(array):.4f}")

def summarize_tensor_stats(name, tensor):
    print(f"[Torch] {name} -> min: {tensor.min().item():.4f}, max: {tensor.max().item():.4f}, mean: {tensor.mean().item():.4f}")

# ---- Load Real NPZ File ----
def load_npz_features(npz_path):
    print(f"\nLoading: {npz_path}")
    data = np.load(npz_path)
    features = data['features']
    labels = data['labels']
    print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    summarize_numpy_stats("Loaded features", features)
    return features, labels

# ---- Compute and Check Tangent Kernel ----
def compute_tangent_features(F_np):
    F_tensor = torch.from_numpy(F_np).float()
    F_T = F_tensor.T.to("cuda:0")
    F_device = F_tensor.to("cuda:0")
    n = F_tensor.shape[0]
    A_tensor = torch.matmul(F_T, F_device) / n
    summarize_tensor_stats("A = F^T F / n", A_tensor)

    FA_tensor = torch.matmul(F_device, A_tensor)
    summarize_tensor_stats("F @ A", FA_tensor)

    FA_np = FA_tensor.cpu().numpy()
    summarize_numpy_stats("F @ A (numpy)", FA_np)
    return A_tensor.cpu().numpy(), FA_np

# ---- Process All Files in Folder ----
def process_all_npz_files(folder_path):
    npz_files = sorted(glob.glob(os.path.join(folder_path, "*.npz")))
    if not npz_files:
        print(f"No .npz files found in {folder_path}")
        return

    print(f"Found {len(npz_files)} .npz files in {folder_path}")

    for i, file_path in enumerate(npz_files):
        print(f"\n[{i+1}/{len(npz_files)}] Processing {file_path}")
        features, labels = load_npz_features(file_path)
        A, tangent_features = compute_tangent_features(features)

# ---- Run on Folder ----
folder_path = "/scratch/bowenxi/dit/neural_tangent_kernel/feature_swin_b/auto_ML_CA_1/auto_ML_CA/"
process_all_npz_files(folder_path)


Found 8 .npz files in /scratch/bowenxi/dit/neural_tangent_kernel/feature_swin_b/auto_ML_CA_1/auto_ML_CA/

[1/8] Processing /scratch/bowenxi/dit/neural_tangent_kernel/feature_swin_b/auto_ML_CA_1/auto_ML_CA/increment_1_train_tangent.npz

Loading: /scratch/bowenxi/dit/neural_tangent_kernel/feature_swin_b/auto_ML_CA_1/auto_ML_CA/increment_1_train_tangent.npz
Features shape: (2305167, 1024), Labels shape: (2305167,)
[NumPy] Loaded features -> min: -25072842.0000, max: 22132024.0000, mean: 81258.4375
[Torch] A = F^T F / n -> min: -108746206347264.0000, max: 116082505416704.0000, mean: 6772740608.0000


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.79 GiB. GPU 0 has a total capacity of 79.20 GiB of which 3.15 GiB is free. Process 171519 has 57.55 GiB memory in use. Including non-PyTorch memory, this process has 18.49 GiB memory in use. Of the allocated memory 17.62 GiB is allocated by PyTorch, and 18.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)